In [1]:
#Importamos librerias utiles
import pandas as pd
import numpy as np
import math
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

#leemos nuestros datos
pico = pd.read_csv('DatosZonaPico.csv',sep=',')
Pico = pd.DataFrame(pico)
cachete = pd.read_csv('DatosZonaCachete.csv',sep=',')
Cachete = pd.DataFrame(cachete)
cuello = pd.read_csv('DatosZonaCuello.csv',sep=',')
Cuello = pd.DataFrame(cuello)

#dado que la columna Var1 no aporta datos utiles, la descartamos
Cachete = Cachete.drop(columns = ['Var1'])
Cuello = Cuello.drop(columns = ['Var1'])
Pico = Pico.drop(columns = ['Var1'])

#mostramos las primeras 10 filas de un dataframe
Cachete.head(10)

,media,varianza,asimetria,curtosis,entropia,contraste,correlacion,energia,homogeneidad
0,83.890625,9466.631127,1.698557,7.044050,0.851806,0.013920,0.994203,0.901224,0.997731
1,83.867188,59045.166605,4.191150,23.352161,0.967662,0.007542,0.994359,0.902242,0.998310
2,69.968750,10482.210784,1.211034,3.279414,0.999956,0.003339,0.993949,0.917516,0.998834
3,33.515625,2414.705637,2.491578,13.102371,0.935456,0.007190,0.992016,0.959531,0.998659
4,31.531250,2021.606863,1.489232,4.374694,0.972305,0.006786,0.989733,0.961862,0.998876
5,100.101562,16955.244547,1.141830,3.560725,0.984047,0.008298,0.995753,0.883347,0.998041
6,129.976562,27044.407292,1.089453,2.849137,0.851806,0.018132,0.995152,0.850431,0.997510
7,88.371094,14708.289200,1.655309,5.201897,0.908910,0.015441,0.993368,0.896396,0.998130
8,93.660156,9485.087975,1.071074,3.527071,0.805028,0.011511,0.994621,0.890437,0.998041
9,108.781250,17184.665686,1.345566,4.095803,0.846354,0.017259,0.994185,0.873381,0.997877


In [2]:
#Mostramos la informacion de cada dataframe
Cachete.info()
Pico.info()
Cuello.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   media         50 non-null     float64
 1   varianza      50 non-null     float64
 2   asimetria     50 non-null     float64
 3   curtosis      50 non-null     float64
 4   entropia      50 non-null     float64
 5   contraste     50 non-null     float64
 6   correlacion   50 non-null     float64
 7   energia       50 non-null     float64
 8   homogeneidad  50 non-null     float64
dtypes: float64(9)
memory usage: 3.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   media         50 non-null     float64
 1   varianza      50 non-null     float64
 2   asimetria     50 non-null     float64
 3   curtosis      50 non-null     float64
 4   entropia      50 non

In [3]:
#Definimos nuestros conjuntos de entrenamiento y validacion con el 70% entrenamiento y 30% evaluacion
#train se construye a partir de la funcion sample() donde frac es el porcentaje de los datos
#random_state es la semilla del numero aleatorio, esta funcion por default tiene desactivado
#el ingreso de datos por reemplazo, es decir no se pueden repetir filas
#el conjunto test se construye a partir del train quitando todos los indices que tiene train,
#es decir hace una operacion de conjuntos del conjunto universo Spam resta el conjunto train 
#y guarda lo que resta en test
trainPico = Pico.sample(frac=0.7,random_state=0)
testPico = Pico.drop(trainPico.index)

trainCachete = Cachete.sample(frac=0.7,random_state=0)
testCachete = Cachete.drop(trainCachete.index)

trainCuello = Cuello.sample(frac=0.7,random_state=0)
testCuello = Cuello.drop(trainCuello.index)

#reseteamos los indices con la funcion reset_index. drop=True lo usamos para evitar que se
#anada una columna del indice anterior
trainPico.reset_index(drop=True,inplace=True)
testPico.reset_index(drop=True,inplace=True)

trainCachete.reset_index(drop=True,inplace=True)
testCachete.reset_index(drop=True,inplace=True)

trainCuello.reset_index(drop=True,inplace=True)
testCuello.reset_index(drop=True,inplace=True)

#Nuestro conjunto de datos debe ser un conjunto de las particiones recien hechas. Para ello definimos
#un nuevo dataframe llamado Test
frames = [testCachete, testCuello, testPico]
frames
Test = pd.concat(frames)
Test.reset_index(drop=True,inplace=True)


#Mostramos un dataframe recien creado
Test.head(10)

,media,varianza,asimetria,curtosis,entropia,contraste,correlacion,energia,homogeneidad
0,83.890625,9466.631127,1.698557,7.044050,0.851806,0.013920,0.994203,0.901224,0.997731
1,33.515625,2414.705637,2.491578,13.102371,0.935456,0.007190,0.992016,0.959531,0.998659
2,129.976562,27044.407292,1.089453,2.849137,0.851806,0.018132,0.995152,0.850431,0.997510
3,108.781250,17184.665686,1.345566,4.095803,0.846354,0.017259,0.994185,0.873381,0.997877
4,41.867188,9897.197978,3.503589,15.631245,0.935456,0.013032,0.993885,0.950004,0.998641
5,37.195312,1874.989154,1.209834,3.970218,0.912999,0.009031,0.989789,0.955148,0.998696
6,125.851562,33358.456311,1.815782,5.696459,0.857148,0.013286,0.996401,0.855026,0.997679
7,70.285156,8604.581112,1.517928,4.998017,0.912999,0.013647,0.992761,0.916647,0.997825
8,96.023438,20198.117096,1.389435,3.795411,0.994666,0.012060,0.993957,0.887892,0.998004
9,90.093750,21156.775490,1.918872,6.027364,0.945373,0.013150,0.994834,0.894740,0.998095


### Un poco de teoría

El clasificador naive Bayes es una forma de aprendizaje supervisado. El nombre de naive o ingenuo proviene de que este tipo de clasificadores se supone que cada par de caracteristicas es condicionalmente independiente entre sí.

Usando el teorema de Bayes es posible deducir que para cada clase $y$ y un vector de características $\vec{x} = x_1 , x_2 ,..., x_n$ tenemos que:

$$P(y| \vec{x}) \approx P(y)\prod_{i=1}^{n}P(x_i | y)$$
<br>
Donde asumimos que cada probabilidad $P(x_i | y)$ sigue una distribucion bien definida. $P(y)$ sera la probabilidad a priori.
De esta manera para cada columna de cada dataframe asumiremos que siguen una distribucion de probabilidad Gausiana o normal dada por:

$$P(x_i | y) = \frac{1}{\sqrt{2 \pi \sigma_{y}^{2}}} exp \left( - \frac{(x_y - \mu_y )^2}{2 \sigma_{y}^{2}}  \right)$$
<br>
El valor estimado $\hat{y}$ vendrá dado por:
<br>
$$\hat{y} = argmax_y P(y)\prod_{i=1}^{n}P(x_i | y)$$
<br>
Dado que asumimos que todas las clases (columnas de la tabla) siguen una distribución normal, debemos calcular $\sigma_{y}^{2}$ y $\mu_y$ para cada una de ellas, para posteriormente calcular $P(x_i | y)$ y finalmente obtener $P(y| \vec{x})$

### Observación de las Imágenes

Notemos que se dividió la imagen en tres regiones: Pico, Cachete y Cuello. Cada imagen contiene dichas regiones por lo que la probabilidad a priori de cada una de las tres clases de interés es $P(y)=\frac{1}{3}$ 

In [4]:
#calculamos la media de cada columna para cada dataframe
mediaPico = trainPico.mean(axis=0)
mediaCuello = trainCuello.mean(axis=0)
mediaCachete = trainCachete.mean(axis=0)

#calculamos la varianza de cada columna para cada dataframe
varianzaPico = trainPico.var(axis=0)
varianzaCuello = trainCuello.var(axis=0)
varianzaCachete = trainCachete.var(axis=0)

### Clasificacion

In [5]:
#Convertimos el dataframe Test a un array numpy
arrayTest = Test.to_numpy()

#Imprimimos es array reciencreado
print(arrayTest)

[[8.38906250e+01 9.46663113e+03 1.69855712e+00 7.04404984e+00
  8.51806189e-01 1.39198388e-02 9.94202731e-01 9.01224202e-01
  9.97730786e-01]
 [3.35156250e+01 2.41470564e+03 2.49157769e+00 1.31023713e+01
  9.35455765e-01 7.19003820e-03 9.92016297e-01 9.59530528e-01
  9.98658658e-01]
 [1.29976562e+02 2.70444073e+04 1.08945275e+00 2.84913682e+00
  8.51806189e-01 1.81324216e-02 9.95151889e-01 8.50431400e-01
  9.97509680e-01]
 [1.08781250e+02 1.71846657e+04 1.34556624e+00 4.09580305e+00
  8.46354101e-01 1.72589095e-02 9.94184673e-01 8.73381398e-01
  9.97876506e-01]
 [4.18671875e+01 9.89719798e+03 3.50358939e+00 1.56312448e+01
  9.35455765e-01 1.30322378e-02 9.93884971e-01 9.50003568e-01
  9.98641066e-01]
 [3.71953125e+01 1.87498915e+03 1.20983377e+00 3.97021767e+00
  9.12999214e-01 9.03098855e-03 9.89789239e-01 9.55147650e-01
  9.98695641e-01]
 [1.25851562e+02 3.33584563e+04 1.81578203e+00 5.69645915e+00
  8.57148437e-01 1.32858381e-02 9.96401209e-01 8.55025996e-01
  9.97679166e-01]
 [7.02

In [6]:
#Creamos matrices auxiliares para almacenar P(x_i | y)
arrayAuxPico = np.zeros(arrayTest.shape,dtype=float)
arrayAuxCuello = np.zeros(arrayTest.shape,dtype=float)
arrayAuxCachete = np.zeros(arrayTest.shape,dtype=float)

constante = 1/(math.sqrt(2*math.pi))
constante

0.3989422804014327

In [7]:
for columnas in range(arrayTest.shape[1]):
    for filas in range(arrayTest.shape[0]):
        Ppico = constante * (1/math.sqrt(pow(varianzaPico[columnas],2))) * (math.exp(-((pow((arrayTest[filas][columnas]-mediaPico[columnas]),2))/(2*pow(varianzaPico[columnas],2)))))
        arrayAuxPico[filas][columnas] = Ppico
        Pcuello = constante * (1/math.sqrt(pow(varianzaCuello[columnas],2))) * (math.exp(-((pow((arrayTest[filas][columnas]-mediaCuello[columnas]),2))/(2*pow(varianzaCuello[columnas],2)))))
        arrayAuxCuello[filas][columnas] = Pcuello
        Pcachete = constante * (1/math.sqrt(pow(varianzaCachete[columnas],2))) * (math.exp(-((pow((arrayTest[filas][columnas]-mediaCachete[columnas]),2))/(2*pow(varianzaCachete[columnas],2)))))
        arrayAuxCachete[filas][columnas] = Pcachete

In [8]:
pred = pd.DataFrame(arrayAuxPico)
pred

,0,1,2,3,4,5,6,7,8
0,0.000521,9.057983e-09,5.063452e-01,0.017293,2.447227e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,0.000522,9.057983e-09,4.039946e-01,0.016715,8.167881e+00,0.000000e+00,0.000000e+00,1.145613e-46,0.0
2,0.000519,9.057982e-09,2.816845e-01,0.017004,2.447227e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.0
3,0.000520,9.057983e-09,3.906474e-01,0.017148,6.523371e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.0
4,0.000522,9.057983e-09,5.961567e-02,0.016147,8.167881e+00,0.000000e+00,0.000000e+00,2.007090e-01,0.0
5,0.000522,9.057983e-09,3.332969e-01,0.017136,2.475700e+01,0.000000e+00,0.000000e+00,2.701044e-19,0.0
6,0.000519,9.057981e-09,5.254861e-01,0.017262,7.992286e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.0
7,0.000522,9.057983e-09,4.558552e-01,0.017222,2.475700e+01,0.000000e+00,0.000000e+00,4.247623e-259,0.0
8,0.000521,9.057982e-09,4.083440e-01,0.017118,3.663481e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.0
9,0.000521,9.057982e-09,5.320549e-01,0.017275,2.683047e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


### Inestabilidad númerica.
Vemos que la moyoria de los valores 5,6,7 y 8 python los detecta como si fueran cero. Sin embargo, los valores reales no son cero, son valores aproximados a $1\times10^{-2250}$ (ingrese algunos a Wolfram). Esto lo consideramos como una inestabilidad numerica.
<br>
Para evitar este problema calculamos la siguiente ecuacion:
<br>
$$ln(P(x_i | y)) = ln \left( \frac{1}{\sqrt{2 \pi \sigma_{y}^{2}}} exp \left( - \frac{(x_y - \mu_y )^2}{2 \sigma_{y}^{2}}  \right)\right)$$
<br>
donde $ln(x)$ es el logaritmo natural de x. Realizando algebra sencilla encontramos que la ecuacion anterior es:
$$ln(P(x_i | y)) = \left( \frac{-(\mu_y - x_y)^2}{2\sigma ^2} + ln\left( \frac{1}{\sigma}\right) - \frac{ln(2\pi)}{2} \right)$$
<br>
Por otro lado, veamos que:
<br>
$$ln(P(y| \vec{x})) \approx ln(P(y)\prod_{i=1}^{n}P(x_i | y))$$
<br>
Por propiedades de los logaritmos resulta ser:
<br>
$$ln(P(y| \vec{x})) \approx ln(P(y)) + \sum_{i=1}^{n}ln(P(x_i | y))$$
<br>

Dado que la predicción $\hat{y}$ escoge el valor maximo, aplicar el logaritmo natural a $P(y| \vec{x})$ no aporta información ya que el logaritmo es una función creciente en el intervalo $(0,\infty)$. Por lo que nuestra predicción $\hat{y}$ tambien será:
<br>
$$\hat{y} = argmax_y \left( ln(P(y)) + \sum_{i=1}^{n}ln(P(x_i | y))\right)$$
<br>

In [9]:
#Creamos matrices auxiliares para almacenar P(x_i | y)
arrayAuxPico = np.zeros(arrayTest.shape,dtype=float)
arrayAuxCuello = np.zeros(arrayTest.shape,dtype=float)
arrayAuxCachete = np.zeros(arrayTest.shape,dtype=float)

In [10]:
#De lo anterior calculamos los logaritmos de P(x_i| y) 
#y guardamos los resultados en los arrays Auxiliares segun sea el caso
for columnas in range(arrayTest.shape[1]):
    for filas in range(arrayTest.shape[0]):
        Ppico = (-1/2)*((pow((-arrayTest[filas][columnas]+mediaPico[columnas]),2))/(pow(varianzaPico[columnas],2)))+(math.log((1/varianzaPico[columnas])))-((1/2)*math.log(2*math.pi))
        arrayAuxPico[filas][columnas] = Ppico
        Pcuello = (-1/2)*(((pow((-arrayTest[filas][columnas]+mediaCuello[columnas]),2)))/(pow(varianzaCuello[columnas],2)))+(math.log((1/varianzaCuello[columnas])))-((1/2)*math.log(2*math.pi))
        arrayAuxCuello[filas][columnas] = Pcuello
        Pcachete = (-1/2)*(((pow((-arrayTest[filas][columnas]+mediaCachete[columnas]),2)))/(pow(varianzaCachete[columnas],2)))+(math.log((1/varianzaCachete[columnas])))-((1/2)*math.log(2*math.pi))
        arrayAuxCachete[filas][columnas] = Pcachete

In [11]:
#Comprobamos visualmente que no haya inestabilidad numerica
predPico = pd.DataFrame(arrayAuxPico)
predPico

,0,1,2,3,4,5,6,7,8
0,-7.558960,-18.519619,-0.680537,-4.057443,-3.710215,-8.938733e+05,-1.250495e+04,-1373.162024,-5.209646e+06
1,-7.558414,-18.519619,-0.906354,-4.091420,2.100209,-8.188050e+04,-9.075104e+03,-105.782975,-1.728927e+05
2,-7.563263,-18.519619,-1.266968,-4.074326,-3.710215,-1.844539e+06,-1.416453e+04,-6188.786003,-7.438037e+06
3,-7.560832,-18.519619,-0.939950,-4.065856,-5.032364,-1.619418e+06,-1.247437e+04,-3584.930009,-3.957366e+06
4,-7.558204,-18.519619,-2.819837,-4.126005,2.100209,-7.370118e+05,-1.197243e+04,-1.605899,-2.035768e+05
5,-7.558307,-18.519619,-1.098721,-4.066577,3.209108,-2.176265e+05,-6.145088e+03,-42.755478,-1.165531e+05
6,-7.562729,-18.519620,-0.643432,-4.059262,-2.526693,-7.802863e+05,-1.650640e+04,-5611.006861,-5.694485e+06
7,-7.558384,-18.519619,-0.785580,-4.061547,3.209108,-8.441277e+05,-1.018133e+04,-594.923179,-4.378165e+06
8,-7.559740,-18.519619,-0.895645,-4.067630,-10.214512,-5.825621e+05,-1.209182e+04,-2302.642488,-3.000820e+06
9,-7.559327,-18.519619,-0.631009,-4.058499,0.986953,-7.568929e+05,-1.359765e+04,-1795.478312,-2.400864e+06


In [12]:
#Comprobamos visualmente que no haya inestabilidad numerica
predCuello = pd.DataFrame(arrayAuxCuello)
predCuello

,0,1,2,3,4,5,6,7,8
0,-7.973218,-24.189353,-2.567879,-7.207556,-82.155999,-5.942774e+05,-29.033617,-82.157890,-402359.907342
1,-7.975494,-24.189353,-2.494879,-7.207251,4.102975,-5.632277e+04,-27.725836,-11.158916,-8374.656150
2,-7.972796,-24.189353,-2.644371,-7.207842,-82.155999,-1.220758e+06,-29.609718,-433.672152,-651735.525747
3,-7.972793,-24.189353,-2.610046,-7.207751,-94.094013,-1.072519e+06,-29.022705,-241.443877,-270726.390349
4,-7.974985,-24.189353,-2.445403,-7.207160,4.102975,-4.907313e+05,-28.841882,1.532150,-6043.191450
5,-7.975263,-24.189353,-2.627847,-7.207760,-1.188054,-1.469094e+05,-26.421423,-4.141865,-14511.536092
6,-7.972769,-24.189353,-2.555194,-7.207642,-71.208217,-5.193053e+05,-30.375737,-390.779581,-455225.076764
7,-7.973646,-24.189353,-2.588712,-7.207688,-1.188054,-5.614481e+05,-28.168081,-28.804509,-314055.114486
8,-7.972953,-24.189353,-2.604481,-7.207773,-44.813951,-3.886876e+05,-28.885183,-148.315890,-176700.350432
9,-7.973069,-24.189353,-2.544581,-7.207620,2.265382,-5.038595e+05,-29.416363,-112.013061,-121982.671954


In [13]:
#Comprobamos visualmente que no haya inestabilidad numerica
predCachete = pd.DataFrame(arrayAuxCachete)
predCachete

,0,1,2,3,4,5,6,7,8
0,-8.495574,-21.246694,-1.475472,-5.339823,-90.773489,-284.133681,-9.336465e+03,4.537566,-5.820433e+05
1,-8.495892,-21.246694,-1.446703,-5.340287,2.520106,-9513.788172,-8.391583e+03,-260.507590,-6.697155e+05
2,-8.495868,-21.246694,-1.650908,-5.342614,-90.773489,-7899.567844,-3.183160e+04,-238.507719,-1.298994e+06
3,-8.495663,-21.246694,-1.560909,-5.341519,-105.394279,-5473.012803,-9.038085e+03,-74.040991,-2.647878e+05
4,-8.495793,-21.246694,-1.737969,-5.342051,2.520106,5.184543,-4.792862e+03,-177.455163,-6.215450e+05
5,-8.495846,-21.246694,-1.605672,-5.341619,1.632506,-4374.064599,-8.036690e+04,-220.371065,-7.768480e+05
6,-8.495819,-21.246694,-1.456996,-5.340442,-77.461014,-30.763346,-8.182203e+04,-198.368579,-7.240149e+05
7,-8.495594,-21.246694,-1.513601,-5.340867,1.632506,-146.660064,-7.475235e+02,-9.006069,-3.622968e+05
8,-8.495597,-21.246694,-1.547857,-5.341762,-80.147454,-203.221270,-5.689061e+03,-16.542392,-8.810894e+04
9,-8.495581,-21.246694,-1.444825,-5.340266,-2.732227,-6.812184,-2.281516e+04,-1.917251,-2.014639e+04


In [14]:
#Dado que vamos a sumar las columnas vamos a tener un array resultante de 1 columna x 50 filas
#para cada conjunto de datos
PyxPico = np.zeros((45,1),dtype=float)
PyxCuello = np.zeros((45,1),dtype=float)
PyxCachete = np.zeros((45,1),dtype=float)


In [15]:
#Creamos variables auxiliares que almacenaran las sumas
Pypico = 0
Pycuello = 0
Pycachete = 0
#Ahora calculamos las sumas de todos las columnas para cada array
for filas in range(arrayTest.shape[0]):
    for columnas in range(arrayTest.shape[1]):
        Pypico += arrayAuxPico[filas][columnas]
        Pycuello += arrayAuxCuello[filas][columnas] 
        Pycachete += arrayAuxCachete[filas][columnas]
    #almacenamos los valores de la suma de las columnas para una fila fija en los vectores correspondientes
    PyxPico[filas][0] = Pypico + (1/3)
    PyxCuello[filas][0] = Pycuello + (1/3)
    PyxCachete[filas][0] = Pycachete + (1/3)
    Pypico = 0
    Pycuello = 0
    Pycachete = 0

In [16]:
#creamos un array que contenga las predicciones de acuerdo al valor maximo de entre los tres arrays
#PyxPico, PyxCuello ,PyxCachete. En la segunda columna escribiremos 0,1,2 si la prediccion se 
#trata de Pico, Cuello o Cachete respectivamente
prediccion = np.zeros((45,2),dtype=float)

#creamos variable auxiliar que contendra los valores maximos
#maximo = 0

#iteramos sobre los arrays PyxPico, PyxCuello ,PyxCachete para encontrar el maximo
for filas in range(PyxPico.shape[0]):
    maximo = max(PyxPico[filas][0],PyxCuello[filas][0],PyxCachete[filas][0])
    prediccion[filas][0]=maximo
    if maximo == PyxPico[filas][0]:
        prediccion[filas][1]=0
    elif maximo == PyxCuello[filas][0]:
        prediccion[filas][1]=1
    else:
        prediccion[filas][1]=2

In [17]:
#Convertimos el array a un dataframe y lo gradamos en la variable Prediccion
Prediccion = pd.DataFrame(prediccion)

#cambiamos los headers
Prediccion.rename(columns={0:'ValorMaximo',1:'ClasePredicha'}, inplace=True)

#mostramos el dataframe recien creado
Prediccion

,ValorMaximo,ClasePredicha
0,-5.917864e+05,2.0
1,-6.477374e+04,1.0
2,-1.339091e+06,2.0
3,-2.795146e+05,2.0
4,-4.968392e+05,1.0
5,-1.614943e+05,1.0
6,-8.061797e+05,2.0
7,-3.632346e+05,2.0
8,-9.413421e+04,2.0
9,-4.300921e+04,2.0


In [18]:
#Convertimos los valores de la columna ClasePredicha a string, esto para poder traducir las clases como ya se establecio
Prediccion['ClasePredicha'] = np.where((Prediccion.ClasePredicha == 0.0),'Pico',Prediccion.ClasePredicha)
Prediccion['ClasePredicha'] = np.where((Prediccion.ClasePredicha == 1.0),'Cuello',Prediccion.ClasePredicha)
Prediccion['ClasePredicha'] = np.where((Prediccion.ClasePredicha == 2.0),'Cachete',Prediccion.ClasePredicha)
Prediccion

,ValorMaximo,ClasePredicha
0,-5.917864e+05,2.0
1,-6.477374e+04,1.0
2,-1.339091e+06,2.0
3,-2.795146e+05,2.0
4,-4.968392e+05,1.0
5,-1.614943e+05,1.0
6,-8.061797e+05,2.0
7,-3.632346e+05,2.0
8,-9.413421e+04,2.0
9,-4.300921e+04,2.0


In [27]:
print(mediaPico)
print(mediaCuello)
print(mediaCachete)
print('Varianza \n')
print(varianzaPico)
print(varianzaCuello)
print(varianzaCachete)

media             52.371540
varianza        7718.299094
asimetria          1.935046
curtosis           7.088493
entropia           0.911514
contraste          0.004269
correlacion        0.979430
energia            0.946604
homogeneidad       0.998865
dtype: float64
media              119.207924
varianza        138540.738927
asimetria            4.019806
curtosis            24.690026
entropia             0.933289
contraste            0.004196
correlacion          0.886861
energia              0.942025
homogeneidad         0.998542
dtype: float64
media              82.695647
varianza        23916.298882
asimetria           2.196093
curtosis            9.544620
entropia            0.925352
contraste           0.012915
correlacion         0.993080
energia             0.903802
homogeneidad        0.998178
dtype: float64
Varianza 

media           7.644613e+02
varianza        4.404317e+07
asimetria       7.496397e-01
curtosis        2.306930e+01
entropia        1.604547e-02
contraste       